In [3]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from scipy import stats

#functions
#import wrangle as wr
#import explore as exp
#import model as mo


warnings.filterwarnings("ignore")

#evaluate
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.feature_selection import f_regression 
from statsmodels.formula.api import ols
import sklearn.preprocessing

#feature engineering
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# modeling methods
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

In [4]:
aac_intake = pd.read_csv('Austin_Animal_Center_Intakes.csv')

In [5]:
aac_intake.head(2)

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A786884,*Brock,01/03/2019 04:19:00 PM,01/03/2019 04:19:00 PM,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor
1,A706918,Belle,07/05/2015 12:59:00 PM,07/05/2015 12:59:00 PM,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver


In [6]:
aac_outcome = pd.read_csv('Austin_Animal_Center_Outcomes.csv')

In [7]:
aac_outcome.head(2)

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A794011,Chunk,05/08/2019 06:20:00 PM,05/08/2019 06:20:00 PM,05/02/2017,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,07/18/2018 04:02:00 PM,07/18/2018 04:02:00 PM,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown


In [8]:
# merge two csvs
aac = pd.merge(aac_intake, aac_outcome, how='left')
aac.head()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color,Date of Birth,Outcome Type,Outcome Subtype,Sex upon Outcome,Age upon Outcome
0,A786884,*Brock,01/03/2019 04:19:00 PM,01/03/2019 04:19:00 PM,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor,NaN,NaN,NaN,NaN,NaN
1,A706918,Belle,07/05/2015 12:59:00 PM,07/05/2015 12:59:00 PM,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver,NaN,NaN,NaN,NaN,NaN
2,A724273,Runster,04/14/2016 06:43:00 PM,04/14/2016 06:43:00 PM,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White,NaN,NaN,NaN,NaN,NaN
3,A665644,NaN,10/21/2013 07:59:00 AM,10/21/2013 07:59:00 AM,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico,NaN,NaN,NaN,NaN,NaN
4,A682524,Rio,06/29/2014 10:38:00 AM,06/29/2014 10:38:00 AM,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,NaN,NaN,NaN,NaN,NaN


In [9]:
aac.head(2).T

,0,1
Animal ID,A786884,A706918
Name,*Brock,Belle
DateTime,01/03/2019 04:19:00 PM,07/05/2015 12:59:00 PM
MonthYear,01/03/2019 04:19:00 PM,07/05/2015 12:59:00 PM
Found Location,2501 Magin Meadow Dr in Austin (TX),9409 Bluegrass Dr in Austin (TX)
Intake Type,Stray,Stray
Intake Condition,Normal,Normal
Animal Type,Dog,Dog
Sex upon Intake,Neutered Male,Spayed Female
Age upon Intake,2 years,8 years


In [10]:
aac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124120 entries, 0 to 124119
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Animal ID         124120 non-null  object
 1   Name              85069 non-null   object
 2   DateTime          124120 non-null  object
 3   MonthYear         124120 non-null  object
 4   Found Location    124120 non-null  object
 5   Intake Type       124120 non-null  object
 6   Intake Condition  124120 non-null  object
 7   Animal Type       124120 non-null  object
 8   Sex upon Intake   124119 non-null  object
 9   Age upon Intake   124120 non-null  object
 10  Breed             124120 non-null  object
 11  Color             124120 non-null  object
 12  Date of Birth     5 non-null       object
 13  Outcome Type      5 non-null       object
 14  Outcome Subtype   4 non-null       object
 15  Sex upon Outcome  5 non-null       object
 16  Age upon Outcome  5 non-null       obj

drop:
- Name (unnecessary)
- MonthYear (repeat)
- Found Location (unnecessary)


encode:
- Intake Type
- Intake Condition
- Animal Type
- Sex upon Intake
- Age upon Intake
- Breed
- Color
- Outcome Type
- Sex upon Outcome
- Age upon Outcome

In [11]:
aac.isnull().sum()

Animal ID                0
Name                 39051
DateTime                 0
MonthYear                0
Found Location           0
Intake Type              0
Intake Condition         0
Animal Type              0
Sex upon Intake          1
Age upon Intake          0
Breed                    0
Color                    0
Date of Birth       124115
Outcome Type        124115
Outcome Subtype     124116
Sex upon Outcome    124115
Age upon Outcome    124115
dtype: int64

In [14]:
aac.rename(columns={"Name": "name", "Animal ID": "animal_id", "Found Location": "found_loc", "Intake Type": "intake_type", "Intake Condition": "intake_cond", "Animal Type": "animal_type", "Sex upon Intake": "intake_sex", "Age upon Intake": "intake_age", "Breed": "breed", "Color": "color", "Date of Birth": "dob", "Outcome Type": "outcome_type", "Outcome Subtype": "outcome_subtype", "Sex upon Outcome": "outcome_sex", "Age upon Outcome": "outcome_age"})

,animal_id,name,DateTime,MonthYear,found_loc,intake_type,intake_cond,animal_type,intake_sex,intake_age,breed,color,dob,outcome_type,outcome_subtype,outcome_sex,outcome_age
0,A786884,*Brock,01/03/2019 04:19:00 PM,01/03/2019 04:19:00 PM,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor,NaN,NaN,NaN,NaN,NaN
1,A706918,Belle,07/05/2015 12:59:00 PM,07/05/2015 12:59:00 PM,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver,NaN,NaN,NaN,NaN,NaN
2,A724273,Runster,04/14/2016 06:43:00 PM,04/14/2016 06:43:00 PM,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White,NaN,NaN,NaN,NaN,NaN
3,A665644,NaN,10/21/2013 07:59:00 AM,10/21/2013 07:59:00 AM,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico,NaN,NaN,NaN,NaN,NaN
4,A682524,Rio,06/29/2014 10:38:00 AM,06/29/2014 10:38:00 AM,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124115,A828158,Odin,01/08/2021 07:01:00 PM,01/08/2021 07:01:00 PM,2224 E Stassney in Austin (TX),Stray,Normal,Dog,Intact Male,10 months,Rottweiler Mix,Black/Tan,NaN,NaN,NaN,NaN,NaN
124116,A830183,NaN,03/03/2021 05:12:00 PM,03/03/2021 05:12:00 PM,6802 Bryonwood Drive in Austin (TX),Stray,Medical,Dog,Intact Female,1 month,Chihuahua Shorthair,White,NaN,NaN,NaN,NaN,NaN
124117,A830180,Gigi,03/03/2021 04:31:00 PM,03/03/2021 04:31:00 PM,Austin (TX),Owner Surrender,Normal,Dog,Intact Female,9 years,Australian Cattle Dog/Belgian Malinois,Brown Brindle/White,NaN,NaN,NaN,NaN,NaN
124118,A830171,NaN,03/03/2021 05:13:00 PM,03/03/2021 05:13:00 PM,1310 West Howard Lane in Austin (TX),Stray,Normal,Dog,Intact Female,1 year,Plott Hound Mix,Brown Brindle/White,NaN,NaN,NaN,NaN,NaN


In [ ]:
aac[aac['unitcnt'].isnull()]

In [16]:
aac_outcome.isnull().sum()

Animal ID               0
Name                39025
DateTime                0
MonthYear               0
Date of Birth           0
Outcome Type           20
Outcome Subtype     67183
Animal Type             0
Sex upon Outcome        1
Age upon Outcome        5
Breed                   0
Color                   0
dtype: int64

In [17]:
aac_outcome.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124491 entries, 0 to 124490
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Animal ID         124491 non-null  object
 1   Name              85466 non-null   object
 2   DateTime          124491 non-null  object
 3   MonthYear         124491 non-null  object
 4   Date of Birth     124491 non-null  object
 5   Outcome Type      124471 non-null  object
 6   Outcome Subtype   57308 non-null   object
 7   Animal Type       124491 non-null  object
 8   Sex upon Outcome  124490 non-null  object
 9   Age upon Outcome  124486 non-null  object
 10  Breed             124491 non-null  object
 11  Color             124491 non-null  object
dtypes: object(12)
memory usage: 11.4+ MB


In [38]:
# merge two csvs
inner = pd.merge(aac_intake, aac_outcome, how='inner')

In [39]:
inner

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color,Date of Birth,Outcome Type,Outcome Subtype,Sex upon Outcome,Age upon Outcome
0,A722993,Sleepy,06/18/2016 11:23:00 AM,06/18/2016 11:23:00 AM,Leander (TX),Stray,Normal,Dog,Neutered Male,3 months,Plott Hound/Labrador Retriever,Black Brindle/White,02/27/2016,Adoption,Foster,Neutered Male,3 months
1,A746505,Toby,04/25/2017 03:57:00 PM,04/25/2017 03:57:00 PM,7404 Langston Dr in Austin (TX),Stray,Normal,Dog,Intact Male,10 months,Chihuahua Shorthair Mix,Tan/White,06/04/2016,Return to Owner,NaN,Intact Male,10 months
2,A725607,*Jimmy,06/21/2016 06:13:00 PM,06/21/2016 06:13:00 PM,Austin (TX),Owner Surrender,Normal,Cat,Neutered Male,3 months,Domestic Shorthair Mix,Brown Tabby/White,03/19/2016,Adoption,Foster,Neutered Male,3 months
3,A749461,*Lars,05/18/2017 03:04:00 PM,05/18/2017 03:04:00 PM,Augusta Bend in Travis (TX),Stray,Normal,Cat,Spayed Female,3 years,Domestic Shorthair Mix,Orange Tabby,05/15/2014,Transfer,Snr,Spayed Female,3 years
4,A792353,NaN,04/09/2019 03:58:00 PM,04/09/2019 03:58:00 PM,3001 S Fm 973 in Austin (TX),Stray,Sick,Cat,Unknown,2 years,Domestic Shorthair Mix,Orange Tabby,04/09/2017,Died,Enroute,Unknown,2 years


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Animal ID         5 non-null      object
 1   Name              4 non-null      object
 2   DateTime          5 non-null      object
 3   MonthYear         5 non-null      object
 4   Found Location    5 non-null      object
 5   Intake Type       5 non-null      object
 6   Intake Condition  5 non-null      object
 7   Animal Type       5 non-null      object
 8   Sex upon Intake   5 non-null      object
 9   Age upon Intake   5 non-null      object
 10  Breed             5 non-null      object
 11  Color             5 non-null      object
 12  Date of Birth     5 non-null      object
 13  Outcome Type      5 non-null      object
 14  Outcome Subtype   4 non-null      object
 15  Sex upon Outcome  5 non-null      object
 16  Age upon Outcome  5 non-null      object
dtypes: object(17)
memory

In [25]:
df = df.rename(columns={"Name": "name", "DateTime": "datetime", "MonthYear": "monthyear", "Animal ID": "animal_id", "Found Location": "found_loc", "Intake Type": "intake_type", "Intake Condition": "intake_cond", "Animal Type": "animal_type", "Sex upon Intake": "intake_sex", "Age upon Intake": "intake_age", "Breed": "breed", "Color": "color", "Date of Birth": "dob", "Outcome Type": "outcome_type", "Outcome Subtype": "outcome_subtype", "Sex upon Outcome": "outcome_sex", "Age upon Outcome": "outcome_age"})

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   animal_id        5 non-null      object
 1   name             4 non-null      object
 2   datetime         5 non-null      object
 3   monthyear        5 non-null      object
 4   found_loc        5 non-null      object
 5   intake_type      5 non-null      object
 6   intake_cond      5 non-null      object
 7   animal_type      5 non-null      object
 8   intake_sex       5 non-null      object
 9   intake_age       5 non-null      object
 10  breed            5 non-null      object
 11  color            5 non-null      object
 12  dob              5 non-null      object
 13  outcome_type     5 non-null      object
 14  outcome_subtype  4 non-null      object
 15  outcome_sex      5 non-null      object
 16  outcome_age      5 non-null      object
dtypes: object(17)
memory usage: 720.0+ byte

In [28]:
df[df['outcome_type'].isnull()]

,animal_id,name,datetime,monthyear,found_loc,intake_type,intake_cond,animal_type,intake_sex,intake_age,breed,color,dob,outcome_type,outcome_subtype,outcome_sex,outcome_age


In [29]:
df

,animal_id,name,datetime,monthyear,found_loc,intake_type,intake_cond,animal_type,intake_sex,intake_age,breed,color,dob,outcome_type,outcome_subtype,outcome_sex,outcome_age
0,A722993,Sleepy,06/18/2016 11:23:00 AM,06/18/2016 11:23:00 AM,Leander (TX),Stray,Normal,Dog,Neutered Male,3 months,Plott Hound/Labrador Retriever,Black Brindle/White,02/27/2016,Adoption,Foster,Neutered Male,3 months
1,A746505,Toby,04/25/2017 03:57:00 PM,04/25/2017 03:57:00 PM,7404 Langston Dr in Austin (TX),Stray,Normal,Dog,Intact Male,10 months,Chihuahua Shorthair Mix,Tan/White,06/04/2016,Return to Owner,NaN,Intact Male,10 months
2,A725607,*Jimmy,06/21/2016 06:13:00 PM,06/21/2016 06:13:00 PM,Austin (TX),Owner Surrender,Normal,Cat,Neutered Male,3 months,Domestic Shorthair Mix,Brown Tabby/White,03/19/2016,Adoption,Foster,Neutered Male,3 months
3,A749461,*Lars,05/18/2017 03:04:00 PM,05/18/2017 03:04:00 PM,Augusta Bend in Travis (TX),Stray,Normal,Cat,Spayed Female,3 years,Domestic Shorthair Mix,Orange Tabby,05/15/2014,Transfer,Snr,Spayed Female,3 years
4,A792353,NaN,04/09/2019 03:58:00 PM,04/09/2019 03:58:00 PM,3001 S Fm 973 in Austin (TX),Stray,Sick,Cat,Unknown,2 years,Domestic Shorthair Mix,Orange Tabby,04/09/2017,Died,Enroute,Unknown,2 years


In [30]:
aac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124120 entries, 0 to 124119
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Animal ID         124120 non-null  object
 1   Name              85069 non-null   object
 2   DateTime          124120 non-null  object
 3   MonthYear         124120 non-null  object
 4   Found Location    124120 non-null  object
 5   Intake Type       124120 non-null  object
 6   Intake Condition  124120 non-null  object
 7   Animal Type       124120 non-null  object
 8   Sex upon Intake   124119 non-null  object
 9   Age upon Intake   124120 non-null  object
 10  Breed             124120 non-null  object
 11  Color             124120 non-null  object
 12  Date of Birth     5 non-null       object
 13  Outcome Type      5 non-null       object
 14  Outcome Subtype   4 non-null       object
 15  Sex upon Outcome  5 non-null       object
 16  Age upon Outcome  5 non-null       obj

In [41]:
# merge two csvs
dfo = pd.merge(aac_intake, aac_outcome, how='outer')

In [42]:
dfo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 248606 entries, 0 to 248605
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Animal ID         248606 non-null  object
 1   Name              170531 non-null  object
 2   DateTime          248606 non-null  object
 3   MonthYear         248606 non-null  object
 4   Found Location    124120 non-null  object
 5   Intake Type       124120 non-null  object
 6   Intake Condition  124120 non-null  object
 7   Animal Type       248606 non-null  object
 8   Sex upon Intake   124119 non-null  object
 9   Age upon Intake   124120 non-null  object
 10  Breed             248606 non-null  object
 11  Color             248606 non-null  object
 12  Date of Birth     124491 non-null  object
 13  Outcome Type      124471 non-null  object
 14  Outcome Subtype   57308 non-null   object
 15  Sex upon Outcome  124490 non-null  object
 16  Age upon Outcome  124486 non-null  obj

In [43]:
dfo = dfo.rename(columns={"Name": "name", "DateTime": "datetime", "MonthYear": "monthyear", "Animal ID": "animal_id", "Found Location": "found_loc", "Intake Type": "intake_type", "Intake Condition": "intake_cond", "Animal Type": "animal_type", "Sex upon Intake": "intake_sex", "Age upon Intake": "intake_age", "Breed": "breed", "Color": "color", "Date of Birth": "dob", "Outcome Type": "outcome_type", "Outcome Subtype": "outcome_subtype", "Sex upon Outcome": "outcome_sex", "Age upon Outcome": "outcome_age"})

In [44]:
dfo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 248606 entries, 0 to 248605
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   animal_id        248606 non-null  object
 1   name             170531 non-null  object
 2   datetime         248606 non-null  object
 3   monthyear        248606 non-null  object
 4   found_loc        124120 non-null  object
 5   intake_type      124120 non-null  object
 6   intake_cond      124120 non-null  object
 7   animal_type      248606 non-null  object
 8   intake_sex       124119 non-null  object
 9   intake_age       124120 non-null  object
 10  breed            248606 non-null  object
 11  color            248606 non-null  object
 12  dob              124491 non-null  object
 13  outcome_type     124471 non-null  object
 14  outcome_subtype  57308 non-null   object
 15  outcome_sex      124490 non-null  object
 16  outcome_age      124486 non-null  object
dtypes: object(

In [35]:
aac_intake.isnull().sum()

Animal ID               0
Name                39051
DateTime                0
MonthYear               0
Found Location          0
Intake Type             0
Intake Condition        0
Animal Type             0
Sex upon Intake         1
Age upon Intake         0
Breed                   0
Color                   0
dtype: int64

In [36]:
aac_intake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124120 entries, 0 to 124119
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Animal ID         124120 non-null  object
 1   Name              85069 non-null   object
 2   DateTime          124120 non-null  object
 3   MonthYear         124120 non-null  object
 4   Found Location    124120 non-null  object
 5   Intake Type       124120 non-null  object
 6   Intake Condition  124120 non-null  object
 7   Animal Type       124120 non-null  object
 8   Sex upon Intake   124119 non-null  object
 9   Age upon Intake   124120 non-null  object
 10  Breed             124120 non-null  object
 11  Color             124120 non-null  object
dtypes: object(12)
memory usage: 11.4+ MB


In [37]:
aac_outcome.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124491 entries, 0 to 124490
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Animal ID         124491 non-null  object
 1   Name              85466 non-null   object
 2   DateTime          124491 non-null  object
 3   MonthYear         124491 non-null  object
 4   Date of Birth     124491 non-null  object
 5   Outcome Type      124471 non-null  object
 6   Outcome Subtype   57308 non-null   object
 7   Animal Type       124491 non-null  object
 8   Sex upon Outcome  124490 non-null  object
 9   Age upon Outcome  124486 non-null  object
 10  Breed             124491 non-null  object
 11  Color             124491 non-null  object
dtypes: object(12)
memory usage: 11.4+ MB


In [40]:
aac_outcome

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A794011,Chunk,05/08/2019 06:20:00 PM,05/08/2019 06:20:00 PM,05/02/2017,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,07/18/2018 04:02:00 PM,07/18/2018 04:02:00 PM,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,08/16/2020 11:38:00 AM,08/16/2020 11:38:00 AM,08/16/2019,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray
3,A720371,Moose,02/13/2016 05:59:00 PM,02/13/2016 05:59:00 PM,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A674754,NaN,03/18/2014 11:47:00 AM,03/18/2014 11:47:00 AM,03/12/2014,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby
...,...,...,...,...,...,...,...,...,...,...,...,...
124486,A828810,*Rory,03/03/2021 04:20:00 PM,03/03/2021 04:20:00 PM,03/26/2020,Adoption,NaN,Dog,Spayed Female,11 months,Chihuahua Shorthair/American Staffordshire Ter...,Brown/Tan
124487,A791377,Bolt,12/28/2020 03:49:00 PM,12/28/2020 03:49:00 PM,01/14/2019,Return to Owner,NaN,Dog,Neutered Male,1 year,Siberian Husky Mix,White
124488,A829611,Chief,03/03/2021 04:59:00 PM,03/03/2021 04:59:00 PM,02/17/2012,Rto-Adopt,NaN,Cat,Neutered Male,9 years,Domestic Shorthair,Orange Tabby
124489,A829964,NaN,03/03/2021 05:57:00 PM,03/03/2021 05:57:00 PM,11/27/2020,Adoption,NaN,Dog,Neutered Male,3 months,Labrador Retriever,Cream


In [45]:
# merge two csvs
df = pd.merge(aac_intake, aac_outcome, how='right')

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124491 entries, 0 to 124490
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Animal ID         124491 non-null  object
 1   Name              85466 non-null   object
 2   DateTime          124491 non-null  object
 3   MonthYear         124491 non-null  object
 4   Found Location    5 non-null       object
 5   Intake Type       5 non-null       object
 6   Intake Condition  5 non-null       object
 7   Animal Type       124491 non-null  object
 8   Sex upon Intake   5 non-null       object
 9   Age upon Intake   5 non-null       object
 10  Breed             124491 non-null  object
 11  Color             124491 non-null  object
 12  Date of Birth     124491 non-null  object
 13  Outcome Type      124471 non-null  object
 14  Outcome Subtype   57308 non-null   object
 15  Sex upon Outcome  124490 non-null  object
 16  Age upon Outcome  124486 non-null  obj

In [47]:
# remember to change DateTime to Date in the end.
df = df.rename(columns={"Name": "name", "DateTime": "datetime", "MonthYear": "monthyear", "Animal ID": "animal_id", "Found Location": "found_loc", "Intake Type": "intake_type", "Intake Condition": "intake_cond", "Animal Type": "animal_type", "Sex upon Intake": "intake_sex", "Age upon Intake": "intake_age", "Breed": "breed", "Color": "color", "Date of Birth": "dob", "Outcome Type": "outcome_type", "Outcome Subtype": "outcome_subtype", "Sex upon Outcome": "outcome_sex", "Age upon Outcome": "outcome_age"})

In [50]:
pd.to_datetime(df['datetime'], format='%y%m%d')

ValueError: time data '05/08/2019 06:20:00 PM' does not match format '%y%m%d' (match)

In [51]:
pd.to_datetime(df['datetime']).dt.date

0         2019-05-08
1         2018-07-18
2         2020-08-16
3         2016-02-13
4         2014-03-18
             ...    
124486    2021-03-03
124487    2020-12-28
124488    2021-03-03
124489    2021-03-03
124490    2021-03-03
Name: datetime, Length: 124491, dtype: object

In [52]:
# remove time from `datetime` col  and convert dtype to date
df['datetime'] = pd.to_datetime(df['datetime']).dt.date

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124491 entries, 0 to 124490
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   animal_id        124491 non-null  object
 1   name             85466 non-null   object
 2   datetime         124491 non-null  object
 3   monthyear        124491 non-null  object
 4   found_loc        5 non-null       object
 5   intake_type      5 non-null       object
 6   intake_cond      5 non-null       object
 7   animal_type      124491 non-null  object
 8   intake_sex       5 non-null       object
 9   intake_age       5 non-null       object
 10  breed            124491 non-null  object
 11  color            124491 non-null  object
 12  dob              124491 non-null  object
 13  outcome_type     124471 non-null  object
 14  outcome_subtype  57308 non-null   object
 15  outcome_sex      124490 non-null  object
 16  outcome_age      124486 non-null  object
dtypes: object(

In [54]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124491 entries, 0 to 124490
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   animal_id        124491 non-null  object        
 1   name             85466 non-null   object        
 2   datetime         124491 non-null  datetime64[ns]
 3   monthyear        124491 non-null  object        
 4   found_loc        5 non-null       object        
 5   intake_type      5 non-null       object        
 6   intake_cond      5 non-null       object        
 7   animal_type      124491 non-null  object        
 8   intake_sex       5 non-null       object        
 9   intake_age       5 non-null       object        
 10  breed            124491 non-null  object        
 11  color            124491 non-null  object        
 12  dob              124491 non-null  object        
 13  outcome_type     124471 non-null  object        
 14  outcome_subtype  573

drop:
- name
- monthyear
- found_loc
- intake_type
- intake_cond
- intake_sex
- intake_age


In [57]:
dropcols = ['name', 'monthyear', 'found_loc', 'intake_type', 'intake_cond', 'intake_sex', 'intake_age']

In [58]:
df = df.drop(columns= dropcols)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124491 entries, 0 to 124490
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   animal_id        124491 non-null  object        
 1   datetime         124491 non-null  datetime64[ns]
 2   animal_type      124491 non-null  object        
 3   breed            124491 non-null  object        
 4   color            124491 non-null  object        
 5   dob              124491 non-null  object        
 6   outcome_type     124471 non-null  object        
 7   outcome_subtype  57308 non-null   object        
 8   outcome_sex      124490 non-null  object        
 9   outcome_age      124486 non-null  object        
dtypes: datetime64[ns](1), object(9)
memory usage: 10.4+ MB


In [60]:
df[df['outcome_sex'].isnull()]

,animal_id,datetime,animal_type,breed,color,dob,outcome_type,outcome_subtype,outcome_sex,outcome_age
66645,A667395,2013-11-27,Dog,Dachshund,Brown Merle,11/17/2006,Return to Owner,NaN,NaN,7 years


In [61]:
df[df['outcome_age'].isnull()]

,animal_id,datetime,animal_type,breed,color,dob,outcome_type,outcome_subtype,outcome_sex,outcome_age
123983,A829186,2021-02-04,Cat,Domestic Medium Hair,Black/White,02/04/2019,Euthanasia,Suffering,Intact Male,NaN
124351,A829947,2021-02-27,Other,Fox,Gray/Brown,02/27/2020,Euthanasia,Rabies Risk,Unknown,NaN
124367,A829977,2021-02-27,Other,Bat,Brown,02/27/2019,Euthanasia,Rabies Risk,Unknown,NaN
124465,A830138,2021-03-03,Other,Skunk,Black,03/03/2019,Euthanasia,Rabies Risk,Unknown,NaN
124475,A830156,2021-03-03,Other,Raccoon,Black,03/03/2020,Euthanasia,Rabies Risk,Unknown,NaN


In [64]:
df['outcome_age'].value_counts()

1 year       22130
2 years      19016
2 months     15029
3 years       7589
3 months      5943
1 month       5348
4 years       4430
5 years       4100
4 months      4024
5 months      3098
6 months      2997
6 years       2727
8 years       2371
7 years       2345
3 weeks       2124
2 weeks       2033
8 months      1981
10 years      1893
10 months     1810
4 weeks       1803
7 months      1598
9 years       1297
9 months      1276
12 years       921
1 weeks        850
11 months      787
11 years       740
1 week         660
13 years       594
14 years       396
3 days         354
2 days         347
15 years       338
1 day          264
6 days         237
4 days         235
0 years        185
5 days         158
16 years       141
5 weeks        121
17 years        85
18 years        49
19 years        27
20 years        19
-1 years         5
22 years         5
21 years         1
24 years         1
25 years         1
23 years         1
-2 years         1
-3 years         1
Name: outcom

In [65]:
df[df['outcome_age'] == "-1 years"]

,animal_id,datetime,animal_type,breed,color,dob,outcome_type,outcome_subtype,outcome_sex,outcome_age
56696,A757376,2018-10-21,Dog,Miniature Schnauzer Mix,White,11/05/2019,Return to Owner,NaN,Spayed Female,-1 years
69525,A687107,2018-02-28,Dog,Rhod Ridgeback,Red/Brown,03/17/2019,Return to Owner,NaN,Neutered Male,-1 years
87772,A687107,2017-06-25,Dog,Rhod Ridgeback,Red/Brown,03/17/2019,Rto-Adopt,NaN,Neutered Male,-1 years
88121,A687107,2017-10-04,Dog,Rhod Ridgeback,Red/Brown,03/17/2019,Return to Owner,NaN,Neutered Male,-1 years
108482,A753893,2015-07-02,Dog,American Bulldog Mix,White/Brown,07/12/2016,Transfer,Partner,Intact Male,-1 years


In [66]:
df[df['outcome_age'] == "-2 years"]

,animal_id,datetime,animal_type,breed,color,dob,outcome_type,outcome_subtype,outcome_sex,outcome_age
8692,A757376,2017-09-05,Dog,Miniature Schnauzer Mix,White,11/05/2019,Rto-Adopt,NaN,Spayed Female,-2 years


In [67]:
df[df['outcome_age'] == "-3 years"]

,animal_id,datetime,animal_type,breed,color,dob,outcome_type,outcome_subtype,outcome_sex,outcome_age
94126,A687107,2016-02-25,Dog,Rhod Ridgeback,Red/Brown,03/17/2019,Return to Owner,NaN,Neutered Male,-3 years


In [68]:
df['outcome_type'].value_counts()

Adoption           55347
Transfer           36713
Return to Owner    21517
Euthanasia          8381
Died                1156
Rto-Adopt            698
Disposal             568
Missing               69
Relocate              22
Name: outcome_type, dtype: int64

In [69]:
df['outcome_type'].value_counts().sum()

124471